# Opis ELO ocen v šahu

ELO ocena temelji, na predpostavki, da je šahovska uspešnost pri posamezni igri slučajna spremenljivka, ki je porazdeljena normalno.
Recimo, da imamo igralca A in B z ELO ocenama $R_A$ in $R_B$, potem je pričakovana vrednost za zmago igralca:
\begin{align}
E_A = \frac{1}{1 + 10^{\frac{R_B - R_A}{400}}} \, \quad && \quad \, E_B = \frac{1}{1 + 10^{\frac{R_A - R_B}{400}}}
\end{align}

Igralec, ki zmaga dobi točke igralec, ki pa izgubi igro izgubi točke.
Večja kot je razlika $|R_A - R_B|$, več točk dobi igralec, ki je zmagal in več jih izgubi igralec, ki je izgubil. To izračunamo po formulah:
\begin{align}
R^{'}_A = R_A + K(S_A - E_A) \, \quad && \quad \, R^{'}_B = R_B + K(S_A - E_B),
\end{align}

kjer je $S_{A/B} \in \{1, \frac{1}{2}, 0\}$ (zmaga, izenači, izgubi) in $K = 16$ (za šibkejše igralce se ponavadi vzame $K = 32$). 

Primer:
- A ima oceno 2600
- B ima oceno 2300 .

Potem sta:
- E_A = $\frac{1}{1 + 10^{\frac{2300 - 2600}{400}}} = 0.849$
- E_B = $\frac{1}{1 + 10^{\frac{2600 - 2300}{400}}} = 0.151$ 

Če zmaga A, sta novi oceni, torej $S_A$ = 1 in $S_B$ = 0:
- $R^{'}_A = 2600 + 16(1 - 0.849) = 2602$
- $R^{'}_B = 2300 + 16(0 - 0.151) = 2298$

Če izenačita, torej S_A = S_B = \frac{1}{2}:
- $R^{'}_A = 2600 + 16(0.5 - 0.849) = 2594$
- $R^{'}_B = 2300 + 16(0.5 - 0.151) = 2306$ 

Če zmaga B, torej $S_B$ = 1 in $S_A$ = 0:
- $R^{'}_A = 2600 + 16(0 - 0.849) = 2586$
- $R^{'}_B = 2300 + 16(1 - 0.151) = 2314$

# Vrste iger

Obravnavali bomo classical, rapid in blitz(vsaka ima tudi več podvrst, omejil se bom le na glavne). Oznaka $a | b$ pomeni $a$ minut z $b$ sekund dodatka za vsako potezo.
- Classical: $90|30$ za prvih $40$ potez, potem dobita igralca dodatnih $30$ min
- Rapid: $15|10$
- Blitz: $3|2$

In [53]:
import pandas

sahisti = pandas.read_csv("Podatki_sahistov.csv")
programi = pandas.read_csv("Podatki_programov.csv")

sahisti

,Ime,Država,Naziv,Classical ocena,Rapid ocena,Blitz ocena,Leto rojstva,Število iger
0,"Haidary, Hameedullah",AFG,,2192,0,0,1975.0,19
1,"Sharify, Aziz",AFG,FM,2173,0,0,1990.0,34
2,"Farazi, Khaiber",AFG,CM,2128,2136,2159,1984.0,97
3,"Fidaeyee, Habibullah",AFG,CM,2101,0,0,1978.0,31
4,"Hedayat, Suliman",AFG,,2093,0,0,NaN,15
...,...,...,...,...,...,...,...,...
20083,"Bostan, Ion",MDA,,1872,0,0,1991.0,0
20084,"Sarivan, Margareta",MDA,,1872,1782,1824,1986.0,32
20085,"Ungureanu, Radu",MDA,,1869,0,0,1943.0,0
20086,"Dvornicen, Ivan",MDA,,1868,0,0,1992.0,15


In [54]:
programi

,Ime,ELO,Leto izdaje,Zadnja različica,Igre
0,Stockfish,3936,2021.0,2008.0,129221
1,Alliestein,3936,2020.0,2019.0,358
2,Komodo,3912,2021.0,2010.0,74295
3,Rybka,3900,2011.0,2005.0,80628
4,Houdini,3854,2017.0,2010.0,66443
...,...,...,...,...,...
529,Fornax,0,NaN,NaN,30
530,Odonata,0,NaN,NaN,27
531,Admete,0,NaN,NaN,26
532,Anka,0,NaN,NaN,25


# 1. Hipoteza

Poskušali bomo ugotoviti, če države z višjo populacijo proizvedejo več šahistov z višjo oceno(classical, rapid in blitz) in tudi višjim nazivom.

# Nazivi

FIDE oz. Internation Chess Federation je glavna ustanova za organizacjo mednarodnih šahovskih tekmovanj. Kot taka ima svoje nazive, ki jih šahist pridobi, če zadošča pravim pogojem.
Imamo dve vrsti naslovov: odprte(Open) in ženske(Women). Odprti naslov lahko dobi kdor koli, med tem ko je ženski naslov lahko dobimo samo ženske. Naslovi:
- Women Candidate Master(WCM), ELO: $2000$ 
- Women Fide Master(WFM), ELO: $2100$
- Candidate Master(CM), ELO: $2200$
- Women International Master(WIM), ELO: $2200$
- FIDE Master(FM), ELO: $2300$
- Women Grandmaster, ELO: $2300$
- FIDE Master, ELO: $2400$
- Grandmaster, ELO: $2500$

Opomba: ELO, na katerega se navizuje naslov, je minimalena classical ocena, ki jo mora kandidat dobiti, ni pa edini pogoj(samo na tega se bomo osredotočili). Ko enkrat naslov osvojiš, ga obdržiš za vedno(tudi, če ne izpolnjuješ pogojem za pridobitev tega naslova).

In [61]:
from Zajem import poberi_drzave

drzave = poberi_drzave()
classical = sahisti[["Ime", ]